# Challenge Three

In [1]:
import os
import logging
import vertexai
from typing import List, Dict, Any, Optional
from pathlib import Path
from dotenv import load_dotenv

env_path = Path.cwd().parent / '.env'
load_dotenv(env_path)

logger = logging.getLogger(__name__)
logging.basicConfig(filename="app.log", level=logging.INFO)

GCP_PROJECT = os.getenv("GCP_PROJECT")
GCP_REGION = os.getenv("GCP_REGION")
GOOGLE_MAP_KEY = os.getenv("GOOGLE_MAP_KEY")

vertexai.init(project=GCP_PROJECT, location=GCP_REGION)

### Agent Tools

In [2]:
from agent_tools import get_lat_lon_from_address, get_weather_forecast


def get_weather(address: str) -> Optional[List[Dict[str, Any]]]:
    """Takes an address and returns a weather forcast from National Weather Service.

    Args:
        address: The street address or place name to geocode (e.g., "1600 Amphitheatre Parkway, Mountain View, CA").

    Returns:
        A list of dictionaries, where each dictionary represents a forecast
        period (e.g., 'Tonight', 'Thursday'). Returns None if an error occurs.
    """
    try:
        lat, lon = get_lat_lon_from_address(address=address, api_key=GOOGLE_MAP_KEY)
        forecast = get_weather_forecast(lat, lon)
        return forecast
    except Exception as e:
        print(f"Something broke. Good luck fixing:\n{e}")
        return None


### Agent Callbacks

In [3]:
from typing import Optional

from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmRequest, LlmResponse

from agent_tools import is_address_in_us, is_user_query_mean


def user_prompt_log_callback(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """Logs the content of the user's latest prompt.

    Args:
        callback_context: The context of the agent executing the callback.
        llm_request: The request object sent to the LLM.

    Returns:
        LlmResponse or None.
    """
    if llm_request.contents:
        last = llm_request.contents[-1]
        if last.role == "user" and last.parts and last.parts[0].text:
            user_text = last.parts[0].text.strip()
            logger.info(f"[{callback_context.agent_name}] USER >> {user_text}")

    return None


def model_response_log_callback(
    callback_context: CallbackContext, llm_response: LlmResponse
) -> Optional[LlmResponse]:
    """Logs the content of the model's response.

    Args:
        callback_context: The context of the agent executing the callback.
        llm_response: The response object received from the LLM.

    Returns:
        LlmResponse or None.
    """
    if llm_response.content and llm_response.content.parts:
        txt = llm_response.content.parts[0].text
        if txt:
            logger.info(f"[{callback_context.agent_name}] MODEL >> {txt.strip()}")


def user_query_check_callback(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """Performs moderation checks on the user's query.

    Checks for non-US addresses and harmful content. If a check fails,
    it returns a pre-canned LlmResponse to stop further processing.

    Args:
        callback_context: The context of the agent executing the callback.
        llm_request: The request object containing the user's query.

    Returns:
        An LlmResponse to short-circuit the chain if moderation fails,
        otherwise None.
    """
    try:
        last = llm_request.contents[-1]

        if last.role == "user" and last.parts and last.parts[0].text:
            user_text = last.parts[0].text.strip()
            if not is_address_in_us(
                project_id=GCP_PROJECT, location=GCP_REGION, user_query=user_text
            ):
                return LlmResponse(
                    content={
                        "role": "model",
                        "parts": [
                            {
                                "text": "Message contains non-US addresses, "
                                "please only query for US addresses."
                            }
                        ],
                    }
                )

            if is_user_query_mean(
                project_id=GCP_PROJECT, location=GCP_REGION, user_query=user_text
            ):
                return LlmResponse(
                    content={"role": "model", "parts": [{"text": "Be nice."}]}
                )

    except Exception as e:
        logger.error(f"Woops:\n{e}")

    return None


def chained_before_callback(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """A chained 'before' callback that runs multiple checks in sequence.

    It first runs a moderation check. If the moderation check returns a
    response, this function immediately returns it. Otherwise, it proceeds
    to log the user's input.

    Args:
        callback_context: The context of the agent executing the callback.
        llm_request: The request object to be processed.

    Returns:
        An LlmResponse if moderation fails, otherwise None.
    """

    # 1. Moderation check
    moderation_result = user_query_check_callback(callback_context, llm_request)
    if moderation_result is not None:
        return moderation_result

    # 2. Log user input
    user_prompt_log_callback(callback_context, llm_request)

    return None

### Weather Agent

In [4]:
from google.adk.agents import LlmAgent
from vertexai.preview import reasoning_engines
from IPython.display import Markdown, display

weather_agent_with_moderation = LlmAgent(
    name="weather_agent_with_moderation",
    model="gemini-2.0-flash", # Can be a string for Gemini or a LiteLlm object
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather], 
#    before_model_callback=chained_before_callback,
#    after_model_callback=model_response_log_callback
)


### Search Agent

In [5]:
from google.adk.tools import google_search, agent_tool

google_search_agent = LlmAgent(
    name="google_search_agent",
    model="gemini-2.0-flash",
    description="A helpful agent that can search the web for information.",
    instruction="You are a helpful assistant that can search the web for information. "
                "When the user asks a question, use the 'google_search' tool to find the answer.",
    tools=[google_search]
)

### Main Agent

In [6]:

main_agent = LlmAgent(
    name="main_agent",
    model="gemini-2.0-flash",
    description="A helpful agent that can answer questions and use tools.",
    instruction="You are a helpful assistant that can answer questions and use tools. "
                "If the user asks about the weather, use the 'weather_agent_with_moderation' tool. "
                "If the user asks a general question, use the 'google_search_agent' tool.",
    tools=[agent_tool.AgentTool(agent=google_search_agent)],
    sub_agents=[weather_agent_with_moderation]
)

In [7]:
app = reasoning_engines.AdkApp(
    agent=main_agent,
    enable_tracing=False,
)

user_id = "test-user-id"
session = app.create_session(user_id=user_id)

for event in app.stream_query(
    user_id=user_id,
    session_id = session.id,
    message="What's the weather in New York for the next three days? Provide response as markdown table.",
):
  lastevent = event

display(Markdown(lastevent["content"]["parts"][0]["text"]))

Here's the weather forecast for New York for the next three days:

| Day             | Forecast                                                                                                                                | Temperature |
|-----------------|-----------------------------------------------------------------------------------------------------------------------------------------|-------------|
| **This Afternoon**| A slight chance of showers and thunderstorms between 2pm and 5pm, then a chance of showers and thunderstorms. Mostly sunny.           | High near 86F   |
| **Tonight**       | Showers and thunderstorms. Mostly cloudy.                                                                                              | Low around 75F    |
| **Thursday**      | A chance of rain showers before 2pm, then a chance of showers and thunderstorms. Mostly cloudy.                                       | High near 79F   |
| **Thursday Night**| A chance of showers and thunderstorms before 8pm, then a chance of showers and thunderstorms between 8pm and 2am. Mostly cloudy.        | Low around 74F    |
| **Friday**        | A slight chance of rain showers between 8am and 2pm, then a chance of showers and thunderstorms. Mostly cloudy.                        | High near 81F   |
| **Friday Night**  | A chance of showers and thunderstorms before 8pm, then a slight chance of showers and thunderstorms. Mostly cloudy.                   | Low around 73F    |



In [9]:
for event in app.stream_query(
    user_id=user_id,
    session_id = session.id,
    message="What's a hot dog?",
):
  lastevent = event

display(Markdown(lastevent["content"]["parts"][0]["text"]))

A hot dog is a cooked sausage, often a frankfurter or wiener, that is typically served in a long, soft roll. It can be grilled, steamed, or boiled. Common condiments include mustard, ketchup, relish, and onions. "Hot dog" can also refer to someone who performs in a conspicuous or ostentatious manner, especially by performing fancy stunts, or be used as an interjection to express approval or pleasure.
